# Prueba Clase 8-8-24

In [31]:
import pandas as pd
import regex as re

In [32]:
DN = pd.read_csv('dnrpa-robos-recuperos-autos-202201.csv')

## Etapa 1 - Definiciones de Features

                                SUMMARY

- Shape (2676,25)
- Variable Target 'tramite_tipo' 
valores DENUNCIA DE ROBO O HURTO / RETENCION INDEBIDA    2621
        COMUNICACIÓN DE RECUPERO                           55 
- Esta BD solo muestra ENERO
- Top 5 de Pcias con más robos
Buenos Aires                 1756
Ciudad Autónoma de Bs.As.     327
Córdoba                       222
Mendoza                       157
Santa Fe                      109
- Modelos desde 1965 al 2011
- Modelos Importados ó Nacionales
- Top 5 de Marcas mas robadas
VOLKSWAGEN                497
RENAULT                   398
CHEVROLET                 368
FIAT                      367
PEUGEOT                   330



                                ISSUES TECNICOS
- Hay que ordenar la variable 'A-Tipo' ya que tiene valores que son iguales pero expresados distintos
- 'automotor_anio_modelo' pasarlo a Int
- Tratamientos de nulos


                                ISSUES DE NEGOCIO
- 'automotor_origen' Qué es Protocolo 21 ? son muchos 976 registros
- Ultima Fecha de Actualización de la BD ? para saber si BD es vieja y 2021 es un ultimo modelo o la BD es nueva y no hay robos desde hace 3 años
- Nose como descubrir si el mismo auto tiene mas de un tipo de denuncia porque no esta la patente por ej
- Como es que el automotor_tipo_descripcion mas robado es Sedan pero el automotor_modelo_descripcion es Gol y Gol Trend que son autos hatchback y no sedanes ?

In [35]:
DN.head(3)

,tramite_tipo,tramite_fecha,fecha_inscripcion_inicial,registro_seccional_codigo,registro_seccional_descripcion,registro_seccional_provincia,automotor_origen,automotor_anio_modelo,automotor_tipo_codigo,automotor_tipo_descripcion,...,automotor_uso_descripcion,titular_tipo_persona,titular_domicilio_localidad,titular_domicilio_provincia,titular_genero,titular_anio_nacimiento,titular_pais_nacimiento,titular_porcentaje_titularidad,titular_domicilio_provincia_id,titular_pais_nacimiento_id
0,DENUNCIA DE ROBO O HURTO / RETENCION INDEBIDA,2022-01-18,1994-06-03,1173,BERAZATEGUI Nº 2,Buenos Aires,Nacional,1994.0,NaN,FURGON VIDRID.C/ASIE,...,Privado,Física,BERAZATEGUI,BUENOS AIRES,Masculino,1972,ARGENTINA,100,6,ARG
1,DENUNCIA DE ROBO O HURTO / RETENCION INDEBIDA,2022-01-31,2000-10-27,1120,LA MATANZA Nº 03,Buenos Aires,Nacional,2000.0,NaN,SEDAN,...,Privado,Física,LOMAS DEL MIRADOR,BUENOS AIRES,Femenino,1991,ARGENTINA,100,6,ARG
2,DENUNCIA DE ROBO O HURTO / RETENCION INDEBIDA,2022-01-11,1994-04-28,2086,CAPITAL FEDERAL Nº 086,Ciudad Autónoma de Bs.As.,Nacional,1994.0,NaN,FUR.VIDRIADO C/ASIEN,...,Privado,Física,C.AUTONOMA DE BS.AS,C.AUTONOMA DE BS.AS,Masculino,1971,PERU,100,2,NaN


In [36]:
DN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2676 entries, 0 to 2675
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   tramite_tipo                    2676 non-null   object 
 1   tramite_fecha                   2676 non-null   object 
 2   fecha_inscripcion_inicial       2676 non-null   object 
 3   registro_seccional_codigo       2676 non-null   int64  
 4   registro_seccional_descripcion  2676 non-null   object 
 5   registro_seccional_provincia    2676 non-null   object 
 6   automotor_origen                2676 non-null   object 
 7   automotor_anio_modelo           2671 non-null   float64
 8   automotor_tipo_codigo           2601 non-null   object 
 9   automotor_tipo_descripcion      2676 non-null   object 
 10  automotor_marca_codigo          2644 non-null   object 
 11  automotor_marca_descripcion     2676 non-null   object 
 12  automotor_modelo_codigo         26

In [37]:
DN.isna().sum()

tramite_tipo                        0
tramite_fecha                       0
fecha_inscripcion_inicial           0
registro_seccional_codigo           0
registro_seccional_descripcion      0
registro_seccional_provincia        0
automotor_origen                    0
automotor_anio_modelo               5
automotor_tipo_codigo              75
automotor_tipo_descripcion          0
automotor_marca_codigo             32
automotor_marca_descripcion         0
automotor_modelo_codigo            36
automotor_modelo_descripcion        0
automotor_uso_codigo                0
automotor_uso_descripcion           0
titular_tipo_persona                0
titular_domicilio_localidad         2
titular_domicilio_provincia         0
titular_genero                      0
titular_anio_nacimiento             0
titular_pais_nacimiento             0
titular_porcentaje_titularidad      0
titular_domicilio_provincia_id      0
titular_pais_nacimiento_id        126
dtype: int64

In [38]:
DN.shape

(2676, 25)

In [39]:
Columnas_seleccionadas = ['tramite_tipo', 'tramite_fecha', 'registro_seccional_descripcion', 'registro_seccional_provincia', 'automotor_origen', 'automotor_anio_modelo', 'automotor_marca_descripcion', 'automotor_tipo_descripcion', 'automotor_modelo_descripcion' ]

In [40]:
DN2 = DN[Columnas_seleccionadas].copy()

In [41]:
DN2.head(3)

,tramite_tipo,tramite_fecha,registro_seccional_descripcion,registro_seccional_provincia,automotor_origen,automotor_anio_modelo,automotor_marca_descripcion,automotor_tipo_descripcion,automotor_modelo_descripcion
0,DENUNCIA DE ROBO O HURTO / RETENCION INDEBIDA,2022-01-18,BERAZATEGUI Nº 2,Buenos Aires,Nacional,1994.0,RENAULT,FURGON VIDRID.C/ASIE,RENAULT TRAFIC
1,DENUNCIA DE ROBO O HURTO / RETENCION INDEBIDA,2022-01-31,LA MATANZA Nº 03,Buenos Aires,Nacional,2000.0,CHEVROLET,SEDAN,CORSA 3P CITI 1.0 N
2,DENUNCIA DE ROBO O HURTO / RETENCION INDEBIDA,2022-01-11,CAPITAL FEDERAL Nº 086,Ciudad Autónoma de Bs.As.,Nacional,1994.0,RENAULT,FUR.VIDRIADO C/ASIEN,RENAULT TRAFIC


In [42]:
col_rename = {'tramite_tipo':'Tramite', 'tramite_fecha':'T-Fecha', 'registro_seccional_descripcion':'Registro', 'registro_seccional_provincia':'R-Pcia', 'automotor_origen':'A-Origen', 'automotor_anio_modelo':'A-Año', 'automotor_marca_descripcion':'A-Marca', 'automotor_tipo_descripcion':'A-Tipo', 'automotor_modelo_descripcion':'A-Modelo'}

In [43]:
DN2 = DN2.rename(columns=col_rename)

In [44]:
DN2.head(3)

,Tramite,T-Fecha,Registro,R-Pcia,A-Origen,A-Año,A-Marca,A-Tipo,A-Modelo
0,DENUNCIA DE ROBO O HURTO / RETENCION INDEBIDA,2022-01-18,BERAZATEGUI Nº 2,Buenos Aires,Nacional,1994.0,RENAULT,FURGON VIDRID.C/ASIE,RENAULT TRAFIC
1,DENUNCIA DE ROBO O HURTO / RETENCION INDEBIDA,2022-01-31,LA MATANZA Nº 03,Buenos Aires,Nacional,2000.0,CHEVROLET,SEDAN,CORSA 3P CITI 1.0 N
2,DENUNCIA DE ROBO O HURTO / RETENCION INDEBIDA,2022-01-11,CAPITAL FEDERAL Nº 086,Ciudad Autónoma de Bs.As.,Nacional,1994.0,RENAULT,FUR.VIDRIADO C/ASIEN,RENAULT TRAFIC


In [45]:
DN2['Tramite'].value_counts()

Tramite
DENUNCIA DE ROBO O HURTO / RETENCION INDEBIDA    2621
COMUNICACIÓN DE RECUPERO                           55
Name: count, dtype: int64

In [46]:
DN2['T-Fecha'].value_counts()

T-Fecha
2022-01-25    150
2022-01-27    149
2022-01-31    141
2022-01-17    141
2022-01-19    140
2022-01-26    135
2022-01-18    131
2022-01-13    130
2022-01-20    130
2022-01-24    129
2022-01-21    128
2022-01-12    127
2022-01-10    127
2022-01-05    127
2022-01-28    126
2022-01-14    122
2022-01-11    117
2022-01-07    110
2022-01-06    108
2022-01-04    105
2022-01-03    103
Name: count, dtype: int64

In [47]:
DN2['Registro'].value_counts().head(5)

Registro
LA MATANZA Nº 09        42
LA MATANZA Nº 10        41
LOMAS DE ZAMORA Nº 7    35
LA MATANZA Nº 03        33
ADROGUE Nº 3            29
Name: count, dtype: int64

In [48]:
DN2['R-Pcia'].value_counts().head(5)

R-Pcia
Buenos Aires                 1756
Ciudad Autónoma de Bs.As.     327
Córdoba                       222
Mendoza                       157
Santa Fe                      109
Name: count, dtype: int64

In [49]:
DN2['R-Pcia'].value_counts()

R-Pcia
Buenos Aires                 1756
Ciudad Autónoma de Bs.As.     327
Córdoba                       222
Mendoza                       157
Santa Fe                      109
Neuquén                        17
Río Negro                      14
Entre Ríos                      9
Chubut                          8
Corrientes                      8
Tucumán                         7
San Luis                        6
Chaco                           6
Misiones                        6
Jujuy                           4
Formosa                         4
San Juan                        4
Tierra del Fuego                4
Santa Cruz                      3
Salta                           2
Santiago del Estero             1
La Pampa                        1
La Rioja                        1
Name: count, dtype: int64

In [50]:
DN2['A-Año'].value_counts().head(5)

A-Año
2011.0    184
2013.0    164
2012.0    160
2010.0    147
2008.0    119
Name: count, dtype: int64

In [51]:
DN2['A-Marca'].value_counts().head(5)

A-Marca
VOLKSWAGEN    497
RENAULT       398
CHEVROLET     368
FIAT          367
PEUGEOT       330
Name: count, dtype: int64

In [52]:
DN2['A-Modelo'].value_counts()

A-Modelo
GOL 1.6                              40
GOL TREND 1.6                        28
FIORINO FIRE 1242 MPI 8V             26
VOYAGE 1.6                           23
CLASSIC 4 P LS ABS+AIRBAG 1.4N       19
                                     ..
F-14.000                              1
CLIO MIO 5P CONFORT PLUS ABS ABCP     1
KANGOO RL 110CV AA PLC IZQ            1
F-14000                               1
325I                                  1
Name: count, Length: 1427, dtype: int64

In [53]:
DN2['A-Tipo'].value_counts()

A-Tipo
SEDAN 4 PTAS                452
SEDAN 5 PTAS                451
SEDAN 5 PUERTAS             400
SEDAN 4 PUERTAS             292
SEDAN 3 PTAS                178
                           ... 
CAMION CON CAJA METALICA      1
FURGON VIDRIADO               1
FURGON VID.C/ASIENTOS         1
FURGON VID. C/ASIENTOS        1
CAMIONETA                     1
Name: count, Length: 89, dtype: int64

In [54]:
set(DN2['A-Tipo'].unique())

{'3 PTAS',
 'ACOPLADO',
 'BERLINA 3 PTAS',
 'BERLINA 3 PTAS.',
 'BERLINA 3 PUERTAS',
 'BERLINA 3 PUERTAS AA',
 'BERLINA 4 PTAS',
 'BERLINA 4 PUERTAS',
 'BERLINA 5 PTAS',
 'BERLINA 5 PTAS.',
 'BERLINA 5 PUERTAS',
 'CAMION',
 'CAMION C/ HIDROGRUA',
 'CAMION CON CAJA METALICA',
 'CAMIONETA',
 'CH. CABINA 3550.',
 'CHASIS C/C C/CAM P/AUT',
 'CHASIS C/CABINA',
 'CHASIS C/CABINA C/CAJA MET',
 'CHASIS C/CABINA P/CA',
 'CHASIS CON CABINA',
 'COUPE',
 'COUPE 2 PUERTAS',
 'FAMILIAR',
 'FAMILIAR VAN',
 'FOURGON COURT TYPE 6',
 'FUR.VIDRIADO C/ASIEN',
 'FURGON',
 'FURGON  600',
 'FURGON 600',
 'FURGON 800',
 'FURGON BLINDADO',
 'FURGON VID C/ASIENTOS',
 'FURGON VID. C/ASIENT',
 'FURGON VID. C/ASIENTOS',
 'FURGON VID.C/ASIENTOS',
 'FURGON VID/ CON ASIENTOS',
 'FURGON VIDR C/ASIENTOS',
 'FURGON VIDRIADO',
 'FURGON VIDRIADO C/ ASIENTOS',
 'FURGON VIDRIADO C/A',
 'FURGON VIDRIADO C/AS',
 'FURGON VIDRIADO C/ASIENTOS',
 'FURGON VIDRIADO CON ASIENTOS',
 'FURGON VIDRID.C/ASIE',
 'FURGONETA',
 'FURGONETA V

In [55]:
DN2['A-Tipo'] = DN2['A-Tipo'].apply(lambda x: str(x).lower())
DN2['A-Tipo'] = DN2['A-Tipo'].apply(lambda x: x.replace('.',' '))
DN2['A-Tipo'] = DN2['A-Tipo'].apply(lambda x: x.strip())

In [56]:
set(DN2['A-Tipo'].unique())

{'3 ptas',
 'acoplado',
 'berlina 3 ptas',
 'berlina 3 puertas',
 'berlina 3 puertas aa',
 'berlina 4 ptas',
 'berlina 4 puertas',
 'berlina 5 ptas',
 'berlina 5 puertas',
 'camion',
 'camion c/ hidrogrua',
 'camion con caja metalica',
 'camioneta',
 'ch  cabina 3550',
 'chasis c/c c/cam p/aut',
 'chasis c/cabina',
 'chasis c/cabina c/caja met',
 'chasis c/cabina p/ca',
 'chasis con cabina',
 'coupe',
 'coupe 2 puertas',
 'familiar',
 'familiar van',
 'fourgon court type 6',
 'fur vidriado c/asien',
 'furgon',
 'furgon  600',
 'furgon 600',
 'furgon 800',
 'furgon blindado',
 'furgon vid  c/asient',
 'furgon vid  c/asientos',
 'furgon vid c/asientos',
 'furgon vid/ con asientos',
 'furgon vidr c/asientos',
 'furgon vidriado',
 'furgon vidriado c/ asientos',
 'furgon vidriado c/a',
 'furgon vidriado c/as',
 'furgon vidriado c/asientos',
 'furgon vidriado con asientos',
 'furgon vidrid c/asie',
 'furgoneta',
 'furgoneta vidriada',
 'midibus',
 'minibus',
 'pick up',
 'pick up cabina dobl

In [57]:
DN2['A-Tipo-Dupl'] = DN2['A-Tipo']

In [58]:
set(DN2['A-Tipo-Dupl'].unique())

{'3 ptas',
 'acoplado',
 'berlina 3 ptas',
 'berlina 3 puertas',
 'berlina 3 puertas aa',
 'berlina 4 ptas',
 'berlina 4 puertas',
 'berlina 5 ptas',
 'berlina 5 puertas',
 'camion',
 'camion c/ hidrogrua',
 'camion con caja metalica',
 'camioneta',
 'ch  cabina 3550',
 'chasis c/c c/cam p/aut',
 'chasis c/cabina',
 'chasis c/cabina c/caja met',
 'chasis c/cabina p/ca',
 'chasis con cabina',
 'coupe',
 'coupe 2 puertas',
 'familiar',
 'familiar van',
 'fourgon court type 6',
 'fur vidriado c/asien',
 'furgon',
 'furgon  600',
 'furgon 600',
 'furgon 800',
 'furgon blindado',
 'furgon vid  c/asient',
 'furgon vid  c/asientos',
 'furgon vid c/asientos',
 'furgon vid/ con asientos',
 'furgon vidr c/asientos',
 'furgon vidriado',
 'furgon vidriado c/ asientos',
 'furgon vidriado c/a',
 'furgon vidriado c/as',
 'furgon vidriado c/asientos',
 'furgon vidriado con asientos',
 'furgon vidrid c/asie',
 'furgoneta',
 'furgoneta vidriada',
 'midibus',
 'minibus',
 'pick up',
 'pick up cabina dobl

In [59]:
DN2[['A-Tipo','A-Tipo-Dupl']] 

,A-Tipo,A-Tipo-Dupl
0,furgon vidrid c/asie,furgon vidrid c/asie
1,sedan,sedan
2,fur vidriado c/asien,fur vidriado c/asien
3,rural 4 ptas,rural 4 ptas
4,sedan 4 puertas,sedan 4 puertas
...,...,...
2671,furgon,furgon
2672,sedan 3 puertas,sedan 3 puertas
2673,coupe 2 puertas,coupe 2 puertas
2674,sedan 5 puertas,sedan 5 puertas


In [60]:
DN2['A-Tipo-Dupl'] = DN2['A-Tipo-Dupl'].apply(lambda x: x.replace('ptas','puertas'))
DN2['A-Tipo-Dupl'] = DN2['A-Tipo-Dupl'].apply(lambda x: x.replace('fur','furgon'))
DN2['A-Tipo-Dupl'] = DN2['A-Tipo-Dupl'].apply(lambda x: x.replace('pick-up','pick up'))
DN2['A-Tipo-Dupl'] = DN2['A-Tipo-Dupl'].apply(lambda x: x.replace('c/asie','con asientos'))
DN2['A-Tipo-Dupl'] = DN2['A-Tipo-Dupl'].apply(lambda x: x.replace('vidrid','vidriada'))

In [61]:
pd.set_option('display.max_rows', None)

In [62]:
DN2[['A-Tipo','A-Tipo-Dupl']] 

,A-Tipo,A-Tipo-Dupl
0,furgon vidrid c/asie,furgongon vidriada con asientos
1,sedan,sedan
2,fur vidriado c/asien,furgon vidriado con asientosn
3,rural 4 ptas,rural 4 puertas
4,sedan 4 puertas,sedan 4 puertas
5,chasis c/cabina p/ca,chasis c/cabina p/ca
6,3 ptas,3 puertas
7,furgon vidriado c/ asientos,furgongon vidriado c/ asientos
8,transporte de carga,transporte de carga
9,berlina 5 ptas,berlina 5 puertas


In [63]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo']

In [64]:
pd.set_option('display.max_rows', None)

In [65]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: x.replace('c/asie','con asientos'))

In [66]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: x.replace('con asientosn','con asientos'))

In [67]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: x.replace('con asientostos','con asientos'))

In [68]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: x.replace('c/ asientos','con asientos'))

In [69]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: x.replace('c/as','con asientos'))

In [70]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: x.replace('con asientost','con asientos'))

In [71]:
#####   DN2['A-Tipo-Trip'].apply(lambda x: re.sub('(asientosn|conasientosn|pu)', 'puertas', str(x)))

In [72]:
re.search('(( +)|(^))(fur)( +)','vifurdriado furc/asien')  # aca estamos probando antes de meterle Apply

In [73]:
re.search('( +)(pick)(.*)', ' pick')
re.search('( +)(pu(er)?(t)?(a)?|ptas)(( +)|($))', ' put ')
def function_replace(pattern, word_sub, string):
    re.sub(pattern, word_sub, string)

In [298]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: re.sub('(( +)|(^))(fur)( +)', 'furgon ',x))

In [75]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: re.sub('( +)(puerta|puer|pu|ptas|puert|ptas)(( +)|($))', ' puertas ',x))

In [204]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: re.sub('(pick-upia|todo terreno|pick-up c/caja mudancera|pick up|pick-up cabina doble|pick-up cabina simple|pick-up cabina y med|pick-up cabina simpl)', 'pick-up',x))

In [194]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: re.sub('(furgoneta|furgon vidrid con asientos|furgon vidriada|furgon vidriado|furgon vid/ con asientos|furgon vid  con asientos|furgon blindado|furgon  600|furgon 800|furgon vid con asientos|fourgon court type 6|furgon vidriado con asientos|furgon vidriado c/a|furgon 600|furgon vidr con asientos)', 'furgon',x))

In [238]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: re.sub('(otros con caja metalica|otroseta|ch  cabina 3550|chasis c/c c/cam p/aut|otros c/caja met|otros p/ca|chasis con cabina|acoplado|midibus|otros aa|camion|berlina otros|otrosmitorio|3 puertas|semiremolque furgon termico|tractor c/cabina dor|tractor c/ cabina dormitorio|transp  de pasajeros|transporte de carga|sin especificacion|coupe 2 puertas|chasis c/cabina|tractor de carretera|minibus|coupe|semirremolque|utilitario|tractor c/cabina dormitorio)', 'otros',x))

In [268]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: re.sub('(familiar|familiar van|rural 5 puertas van)', 'rural 5 puertas',x))

In [280]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: re.sub('(rural 4/5 puertas|rural 2/otros|rural otros)', 'rural 4 puertas',x))

In [304]:
DN2['A-Tipo-Trip'] = DN2['A-Tipo-Trip'].apply(lambda x: 'sedan 2 puertas' if x=='sedan' else x)

In [306]:
DN2[['A-Tipo-Trip']].value_counts().sort_index()

A-Tipo-Trip       
berlina 4 puertas       1
berlina 4 puertas       2
berlina 5 puertas       4
berlina 5 puertas       4
furgon                200
otros                  65
otros                   3
pick-up               302
rural 4 puertas         2
rural 4 puertas         5
rural 5 puertas        87
rural 5 puertas        83
sedan 2 puertas        24
sedan 2 puertas        30
sedan 4 puertas       292
sedan 4 puertas       452
sedan 5 puertas       400
sedan 5 puertas       451
sedan otros            90
sedan otros           179
Name: count, dtype: int64